Data loading

# Dashboard Design for Merged Music Dataset (Spotify, Billboard, Grammys)

This document outlines a data-driven approach for designing a Power BI dashboard using a dataset created through left joins between Spotify (audio features), Billboard (chart performance), and Grammy Awards (award nominations and categories).

---

## Available Variables

### Spotify Audio Features
- `popularity`: Popularity score assigned by Spotify
- `explicit`: Boolean flag indicating explicit lyrics
- `tempo`, `valence`, `energy`, `danceability`, `acousticness`: Numeric features describing musical properties
- `duration_minutes`: Track duration in minutes
- `track_genre`: Music genre classification

### Grammy Awards Metadata
- `year`: Year of the Grammy edition
- `category`: Award category (e.g., Song of the Year)

### Billboard Chart Data
- `first_chart_date`: Date the song first appeared on the Billboard Hot 100
- `billboard_peak`: Best chart position achieved
- `total_weeks_on_chart`: Number of weeks the song remained on the chart

---

## Analytical Goals and Visualizations

| Goal                                       | Recommended Chart Type                        |
|--------------------------------------------|-----------------------------------------------|
| Explore correlations between features      | Scatter plots (e.g., popularity vs peak rank) |
| Compare genres and musical characteristics | Boxplots or violin plots by `track_genre`     |
| Analyze Grammy award distribution          | Stacked bar chart (songs by year/category)    |
| Map genre to award category relationships  | Heatmap or matrix of `track_genre` vs `category` |
| Identify temporal trends                   | Line charts using `first_chart_date`          |
| Compare multivariate success factors       | Bubble plots (e.g., popularity vs weeks on chart) |
| Measure data source coverage               | Bar chart of counts grouped by source overlap |

---

## Recommended KPIs for Power BI Cards

- Maximum `popularity`
- Highest `total_weeks_on_chart`
- Most common `track_genre` among Grammy-nominated songs
- Count of songs with and without Billboard or Grammy data

---

## Data Join Strategy Summary

- A left join ensures that all Spotify songs are retained
- Billboard and Grammy data are merged only when a `song_name` (and `artist` for Billboard) match
- This design allows the dashboard to reflect both enriched records and those available only in Spotify

---

## Power BI Visualizations to Implement

1. Scatter plot: Popularity vs Billboard peak
2. Violin plot: Popularity distribution by genre
3. Stacked bar chart: Number of Grammy-nominated songs per year
4. Matrix: Track genre vs Grammy category
5. Line chart: Billboard presence over time by genre
6. Bubble chart: Popularity vs total weeks on chart
7. Bar chart: Count of songs by source coverage (Spotify only, +Grammy, +Billboard)
8. KPI cards: Top popularity, longest chart presence, most common genre, data source match summary


In [3]:
import os
from dotenv import load_dotenv
import psycopg2

load_dotenv()


dbname   = os.environ.get("DB_NAME")
user     = os.environ.get("DB_USER")
password = os.environ.get("DB_PASSWORD")
host     = os.environ.get("DB_HOST")
port     = os.environ.get("DB_PORT")

conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)
cur = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS public.grammy_awards (
    year INTEGER,
    title TEXT,
    published_at TEXT,
    updated_at TEXT,
    category TEXT,
    nominee TEXT,
    artist TEXT,
    workers TEXT,
    img TEXT,
    winner BOOLEAN
);
"""
cur.execute(create_table_query)
conn.commit()

csv_file_path = '../data/raw/the_grammy_awards.csv'

copy_query = """
COPY public.grammy_awards FROM STDIN WITH CSV HEADER DELIMITER ',' NULL ''
"""
with open(csv_file_path, 'r', encoding='utf-8') as f:
    cur.copy_expert(copy_query, f)

conn.commit()
cur.close()
conn.close()

print("Data loaded successfully!")


Data loaded successfully!


Extract

In [4]:
import os
import polars as pl
import requests
import pandas as pd
import psycopg2
from dotenv import load_dotenv

def extract_spotify():
    df = pl.read_csv("../data/raw/spotify_dataset.csv")
    df.write_csv("../data/raw/spotify_dataset2.csv")

def extract_grammys():
    load_dotenv()

    dbname = os.getenv("DB_NAME")
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    host = os.getenv("DB_HOST")
    port = os.getenv("DB_PORT")

    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )

    cur = conn.cursor()
    cur.execute("SELECT * FROM public.grammy_awards;")
    columns = [desc[0] for desc in cur.description]
    rows = cur.fetchall()
    cur.close()
    conn.close()

    df = pl.DataFrame(rows, schema=columns)
    df.write_csv("../data/raw/grammy_awards_full.csv")

def extract_billboard():
    url = "https://raw.githubusercontent.com/mhollingshead/billboard-hot-100/main/all.json"
    response = requests.get(url)

    if response.status_code == 200:
        charts = response.json()
        all_data = []

        for chart in charts:
            chart_date = chart["date"]
            for entry in chart["data"]:
                all_data.append({
                    "date": chart_date,
                    "rank": entry.get("this_week"),
                    "title": entry.get("song"),
                    "artist": entry.get("artist"),
                    "last_week": entry.get("last_week"),
                    "peak_position": entry.get("peak_position"),
                    "weeks_on_chart": entry.get("weeks_on_chart")
                })

        df = pd.DataFrame(all_data)
        df.to_csv("../data/raw/billboard_full_chart_data.csv", index=False)
        print("Billboard data extracted successfully!")

    else:
        print(f"Failed to fetch chart data. Status code: {response.status_code}")

extract_spotify()
extract_grammys()
extract_billboard()

C:\Users\57302\AppData\Local\Temp\ipykernel_39636\4080410312.py:36: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(rows, schema=columns)


Billboard data extracted successfully!


trasform

In [3]:
import polars as pl
import os
import re

DATA_RAW = "../data/raw"
DATA_PROCESSED = "../data/processed"

def normalize_text(value):
    if isinstance(value, str):
        value = value.lower()
        value = re.sub(r"[^a-z0-9\s;]", "", value)
        value = re.sub(r"\s+", " ", value)
        return value.strip()
    return value

def transform_spotify():
    df = pl.read_csv(f"{DATA_RAW}/spotify_dataset2.csv")
    df = df.rename({col: col.strip().lower().replace(" ", "_") for col in df.columns})

    df = df.with_columns([
        pl.col("track_name").map_elements(normalize_text, return_dtype=pl.Utf8),
        pl.col("artists").map_elements(normalize_text, return_dtype=pl.Utf8)
    ])

    df = df.select([
        pl.col("track_name").alias("song_name"),
        pl.col("artists").alias("artist"),
        "track_genre",
        "popularity",
        "explicit",
        "tempo",
        "valence",
        "energy",
        "danceability",
        "acousticness",
        "duration_ms"
    ])

    df = df.with_columns([
        (pl.col("duration_ms") / 60000).alias("duration_minutes")
    ])

    df.write_csv(f"{DATA_PROCESSED}/spotify_transformed.csv")

def transform_grammys():
    df = pl.read_csv(f"{DATA_RAW}/grammy_awards_full.csv")
    df = df.rename({col: col.strip().lower().replace(" ", "_") for col in df.columns})

    df = df.with_columns([
        pl.col("nominee").map_elements(normalize_text, return_dtype=pl.Utf8),
        pl.col("artist").map_elements(normalize_text, return_dtype=pl.Utf8),
        pl.col("category").map_elements(normalize_text, return_dtype=pl.Utf8)
    ])

    df = df.filter(
        (pl.col("category").str.contains("song") | pl.col("category").str.contains("record")) &
        ~(pl.col("category").str.contains("album") | pl.col("category").str.contains("artist"))
    )

    df = df.with_columns([
        pl.col("nominee").alias("song_name")
    ])

    df.write_csv(f"{DATA_PROCESSED}/grammys_transformed.csv")

def transform_billboard():
    df = pl.read_csv(f"{DATA_RAW}/billboard_full_chart_data.csv")
    df = df.rename({col: col.strip().lower().replace(" ", "_") for col in df.columns})

    df = df.with_columns([
        pl.col("title").map_elements(normalize_text, return_dtype=pl.Utf8),
        pl.col("artist").map_elements(normalize_text, return_dtype=pl.Utf8)
    ])

    df = df.with_columns([
        pl.col("title").alias("song_name"),
        pl.col("date").str.slice(0, 4).cast(pl.Int64).alias("first_year_on_chart")
    ])

    df.write_csv(f"{DATA_PROCESSED}/billboard_transformed.csv")

def transform_all_datasets():
    transform_spotify()
    transform_grammys()
    transform_billboard()

transform_all_datasets()

Merge

In [4]:
import polars as pl
import os

DATA_PROCESSED = "../data/processed"

def merge_datasets():
    df_spotify = pl.read_csv(f"{DATA_PROCESSED}/spotify_transformed.csv")
    df_grammys = pl.read_csv(f"{DATA_PROCESSED}/grammys_transformed.csv")
    df_billboard = pl.read_csv(f"{DATA_PROCESSED}/billboard_transformed.csv")

    df_spotify = df_spotify.unique()
    df_grammys = df_grammys.unique()
    df_billboard = df_billboard.unique()

    df_spotify = df_spotify.with_columns([
        (pl.col("song_name") + "|" + pl.col("artist")).alias("merge_key")
    ]).sort("popularity", descending=True)

    df_spotify = df_spotify.group_by("merge_key").agg([
        pl.col("song_name").first(),
        pl.col("artist").first(),
        pl.col("track_genre").unique().alias("track_genres"),
        pl.col("popularity").first(),
        pl.col("explicit").first(),
        pl.col("tempo").first(),
        pl.col("valence").first(),
        pl.col("energy").first(),
        pl.col("danceability").first(),
        pl.col("acousticness").first(),
        pl.col("duration_minutes").first()
    ])

    df_spotify = df_spotify.with_columns([
        pl.col("track_genres").list.join(", ").alias("track_genre")
    ]).drop("track_genres")

    df_grammys = df_grammys.sort("year", descending=True)
    df_grammys = df_grammys.group_by("song_name").agg([
        pl.col("year").first(),
        pl.col("category").first(),
        pl.col("published_at").first()
    ])

    df_billboard = df_billboard.with_columns([
        (pl.col("song_name") + "|" + pl.col("artist")).alias("merge_key")
    ])

    df_billboard_grouped = df_billboard.group_by("merge_key").agg([
        pl.col("date").min().alias("first_chart_date"),
        pl.col("rank").min().alias("billboard_peak"),
        pl.col("weeks_on_chart").sum().alias("total_weeks_on_chart"),
        pl.col("first_year_on_chart").min()
    ])

    merged = df_spotify.join(df_grammys, on="song_name", how="left")
    merged = merged.join(df_billboard_grouped, on="merge_key", how="left")

    merged = merged.drop("merge_key")

    columns_to_drop = [
        "duration_ms",
        "nominee",
        "artist_grammy",
        "workers",
        "img",
        "winner",
        "first_year_on_chart",
        "billboard_artist",
        "updated_at",
        "title",
        "published_at"
    ]
    existing_to_drop = [col for col in columns_to_drop if col in merged.columns]
    merged = merged.drop(existing_to_drop)

    merged.write_csv(f"{DATA_PROCESSED}/final_dataset.csv")

merge_datasets()


In [4]:
df = pl.read_csv("../data/processed/final_dataset.csv")
df.head(5)

song_name,artist,popularity,explicit,tempo,valence,energy,danceability,acousticness,duration_minutes,track_genre,year,category,first_chart_date,billboard_peak,total_weeks_on_chart
str,str,i64,bool,f64,f64,f64,f64,f64,f64,str,str,str,str,i64,i64
"""parachute""","""porya hatami""",5,false,68.976,0.0486,0.0892,0.355,0.975,2.93345,"""iranian""",null,null,null,null,null
"""danny trejo""","""plastilina mosh""",25,true,93.984,0.756,0.968,0.771,0.288,4.003333,"""afrobeat""",null,null,null,null,null
"""atak gaya arijit singh""","""arijit singh;rupali moghe""",74,false,80.568,0.508,0.366,0.4,0.87,3.32345,"""pop, pop-film""",null,null,null,null,null
"""places""","""martin solveig;ina wroldsen""",62,false,121.995,0.62,0.854,0.728,0.248,3.379083,"""house, disco, edm""",null,null,null,null,null
"""oh my gawd feat nicki minaj k4…","""mr eazi;major lazer;nicki mina…",45,false,94.018,0.834,0.866,0.82,0.152,3.0,"""dancehall""",null,null,null,null,null


### Coverage of Spotify Songs in Billboard and Grammy Datasets

In [6]:
total_songs = df.select(pl.len()).item()
billboard_count = df.filter(pl.col("billboard_peak").is_not_null()).select(pl.len()).item()
grammy_count = df.filter(pl.col("category").is_not_null()).select(pl.len()).item()

billboard_pct = round((billboard_count / total_songs) * 100, 2)
grammy_pct = round((grammy_count / total_songs) * 100, 2)

print(f"{billboard_pct}% of Spotify songs also appear in the Billboard dataset.")
print(f"{grammy_pct}% were nominated for the Grammys.")

2.76% of Spotify songs also appear in the Billboard dataset.
0.59% were nominated for the Grammys.


In [8]:

# Definir cuántas canciones populares quieres ver
top_n = 30

# Ordenar por popularidad descendente y seleccionar las primeras
most_popular = df.sort("popularity", descending=True)

# Mostrar en notebook
print(most_popular)

# Guardar en CSV
most_popular.write_csv("../data/processed/most_popular.csv")


shape: (78_441, 16)
┌────────────┬───────────┬───────────┬──────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ song_name  ┆ artist    ┆ popularit ┆ explicit ┆ … ┆ category ┆ first_cha ┆ billboard ┆ total_wee │
│ ---        ┆ ---       ┆ y         ┆ ---      ┆   ┆ ---      ┆ rt_date   ┆ _peak     ┆ ks_on_cha │
│ str        ┆ str       ┆ ---       ┆ bool     ┆   ┆ str      ┆ ---       ┆ ---       ┆ rt        │
│            ┆           ┆ i64       ┆          ┆   ┆          ┆ str       ┆ i64       ┆ ---       │
│            ┆           ┆           ┆          ┆   ┆          ┆           ┆           ┆ i64       │
╞════════════╪═══════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ unholy     ┆ sam       ┆ 100       ┆ false    ┆ … ┆ null     ┆ null      ┆ null      ┆ null      │
│ feat kim   ┆ smith;kim ┆           ┆          ┆   ┆          ┆           ┆           ┆           │
│ petras     ┆ petras    ┆           ┆          ┆   ┆          ┆       

In [9]:
df = pl.read_csv("../data/processed/final_dataset.csv")
fully_filled = df.filter(
    pl.all_horizontal([pl.col(c).is_not_null() for c in df.columns])
)

print(f"Number of rows with all columns filled: {fully_filled.shape[0]}")

fully_filled.write_csv("../data/processed/fully_filled_rows.csv")

Number of rows with all columns filled: 89


In [10]:
df = pl.read_csv("../data/processed/fully_filled_rows.csv")
df.head(10)

song_name,artist,popularity,explicit,tempo,valence,energy,danceability,acousticness,duration_minutes,track_genre,year,category,first_chart_date,billboard_peak,total_weeks_on_chart
str,str,i64,bool,f64,f64,f64,f64,f64,f64,str,i64,str,str,i64,i64
"""just the way you are""","""billy joel""",72,false,139.148,0.513,0.453,0.589,0.703,4.842883,"""songwriter, piano, singer-song…",1978,"""song of the year""","""1977-11-12""",3,378
"""lonely boy""","""the black keys""",73,false,166.3,0.607,0.872,0.356,0.00417,3.22755,"""garage, blues, punk, punk-rock""",2012,"""best rock song""","""2011-11-12""",64,210
"""carry on""","""xxxtentacion""",75,true,147.899,0.383,0.197,0.735,0.768,2.162,"""emo""",1997,"""best dance recording""","""2017-09-16""",95,1
"""truth hurts""","""lizzo""",1,true,158.088,0.407,0.622,0.715,0.11,2.88875,"""hip-hop""",2019,"""song of the year""","""2019-05-18""",1,903
"""youre still the one""","""shania twain""",78,false,133.822,0.634,0.494,0.585,0.363,3.536667,"""country""",1998,"""best country song""","""1998-02-14""",2,903
"""tears in heaven""","""eric clapton""",66,false,153.744,0.312,0.242,0.498,0.835,4.529333,"""blues""",1992,"""record of the year""","""1992-02-08""",2,351
"""you make me feel like dancing""","""leo sayer""",60,false,96.071,0.705,0.731,0.734,0.0784,3.688433,"""disco""",1977,"""best rhythm blues song""","""1976-10-23""",1,231
"""good life""","""onerepublic""",74,true,94.988,0.645,0.69,0.634,0.0771,4.221767,"""rock, piano""",2007,"""best rap song""","""2009-11-28""",8,630
"""after the love has gone""","""earth wind fire""",67,false,131.675,0.391,0.443,0.424,0.532,4.634,"""disco""",1979,"""best rhythm blues song""","""1979-07-07""",2,153


In [11]:

# Definir cuántas canciones populares quieres ver
top_n = 20

# Ordenar por popularidad descendente y seleccionar las primeras
most_popular = df.sort("popularity", descending=True)

# Mostrar en notebook
print(most_popular)



shape: (89, 16)
┌───────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ song_name ┆ artist    ┆ popularit ┆ explicit ┆ … ┆ category  ┆ first_cha ┆ billboard ┆ total_wee │
│ ---       ┆ ---       ┆ y         ┆ ---      ┆   ┆ ---       ┆ rt_date   ┆ _peak     ┆ ks_on_cha │
│ str       ┆ str       ┆ ---       ┆ bool     ┆   ┆ str       ┆ ---       ┆ ---       ┆ rt        │
│           ┆           ┆ i64       ┆          ┆   ┆           ┆ str       ┆ i64       ┆ ---       │
│           ┆           ┆           ┆          ┆   ┆           ┆           ┆           ┆ i64       │
╞═══════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ someone   ┆ lewis     ┆ 87        ┆ false    ┆ … ┆ song of   ┆ 2019-05-2 ┆ 1         ┆ 1485      │
│ you loved ┆ capaldi   ┆           ┆          ┆   ┆ the year  ┆ 5         ┆           ┆           │
│ revenge   ┆ xxxtentac ┆ 87        ┆ true     ┆ … ┆ best      ┆ 2017-09-1 

# Merge Process and Final Dataset

## 1. Join Key

The datasets are merged using a common key: `song_name`.  
This field is standardized across datasets as follows:

- From the **Spotify** dataset: derived from the `track_name` column  
- From the **Grammys** dataset: derived from the `nominee` column  
- From the **Billboard** dataset: derived from the `title` column  

During the transformation step, these fields are cleaned by:
- Converting to lowercase
- Removing special characters
- Replacing multiple spaces with a single space
- Trimming leading/trailing spaces

## 2. Merge Logic

The merging is performed in the following order using `left joins`:

1. **Spotify** is used as the base dataset. All songs in this dataset will be preserved.
2. The **Grammys** dataset is joined on `song_name` using a left join.
3. The **Billboard** dataset is then joined using the same key and join type.

## 3. Resulting Structure

The final merged dataset is written as a flat CSV file:  
`/data/processed/final_dataset.csv`

Each row represents a unique song (from Spotify), possibly enriched with:
- Grammy award metadata (if the song won or was nominated)
- Billboard chart performance (if the song entered the charts)

## 4. Example Schema

| song_name     | artist          | track_genre | popularity | tempo | nominee     | category           | grammy_year | billboard_peak | weeks_on_chart | first_year_on_chart |
|---------------|------------------|--------------|------------|--------|--------------|---------------------|--------------|----------------|----------------|----------------------|
| bad guy       | billie eilish    | pop          | 92         | 135.5  | bad guy       | record of the year  | 2019         | 1              | 15             | 2019                 |
| 7 rings       | ariana grande    | pop          | 89         | 130.0  | 7 rings       | record of the year  | 2019         | 2              | 12             | 2019                 |
| sunflower     | post malone      | hip hop      | 87         | 90.0   | sunflower     | record of the year  | 2019         | 3              | 14             | 2019                 |
| unknown song  | unknown          | unknown      | 55         | 120.0  | null          | null                | null         | null           | null           | null                 |

## 5. Purpose

The final dataset is ready for analysis and visualization in BI tools such as Power BI, Looker, or Tableau. It allows you to:

- Analyze the overlap between musical popularity and awards
- Identify trends in genres, artists, or musical features
- Visualize song performance across Spotify, Billboard, and Grammy records


Load

In [2]:
import os
import psycopg2
from dotenv import load_dotenv
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

DATA_PROCESSED = "../data/processed"
CSV_FILENAME = "final_dataset.csv"
CSV_FILE_PATH = f"{DATA_PROCESSED}/{CSV_FILENAME}"

def load_and_store_final_dataset():
    load_dotenv()

    dbname   = os.getenv("DB_NAME")
    user     = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    host     = os.getenv("DB_HOST")
    port     = os.getenv("DB_PORT")

    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()

    # Crear tabla con tipos y orden correcto
    create_table_query = """
    CREATE TABLE IF NOT EXISTS public.music_dataset (
        song_name TEXT,
        artist TEXT,
        popularity INTEGER,
        explicit BOOLEAN,
        tempo FLOAT,
        valence FLOAT,
        energy FLOAT,
        danceability FLOAT,
        acousticness FLOAT,
        duration_minutes FLOAT,
        track_genre TEXT,
        year INTEGER,
        category TEXT,
        first_chart_date TEXT,
        billboard_peak INTEGER,
        total_weeks_on_chart INTEGER
    );
    """
    cur.execute(create_table_query)
    conn.commit()

    # Cargar datos desde el CSV
    copy_query = """
    COPY public.music_dataset (
        song_name, artist, popularity, explicit, tempo, valence, energy,
        danceability, acousticness, duration_minutes, track_genre, year,
        category, first_chart_date, billboard_peak, total_weeks_on_chart
    )
    FROM STDIN WITH CSV HEADER DELIMITER ',' NULL ''
    """
    with open(CSV_FILE_PATH, 'r', encoding='utf-8') as f:
        cur.copy_expert(copy_query, f)

    conn.commit()
    cur.close()
    conn.close()
    print("Final dataset loaded into PostgreSQL.")

    # Subir a Google Drive
    gauth = GoogleAuth()
    gauth.LocalWebserverAuth()
    drive = GoogleDrive(gauth)

    file_to_upload = drive.CreateFile({'title': CSV_FILENAME})
    file_to_upload.SetContentFile(CSV_FILE_PATH)
    file_to_upload.Upload()

    print("Final dataset uploaded to Google Drive.")

load_and_store_final_dataset()


Final dataset loaded into PostgreSQL.
Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=770987404450-glri2v2td4808pcgq31uqii339ednft6.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.
Final dataset uploaded to Google Drive.
